# Cap 3. - Dicionários e Conjuntos
### "Variações de dict" e "Criando subclasses de UserDict" (pag 105 a 108)

```
Data: 15/09/2020
Páginas: 105 a 108
Livro/Edição: Python Fluent, Ed.1
Python: 3.8.2
```

---

### Tópicos
- Variações de dict
    - OrderedDict
    - ChainMap
    - Counter
    - UserDict
- Criando subclasses de UserDict

---

## Variações de Dict

### 3.1. OrderedDict

**Livro:** "Um dicionário que lembra a ordem de inserção. Permite que a iteração pelos itens seja feita em uma ordem previsível."

**Doc:** "Retorna uma subclasse de dict com métodos especializados em rearranjar a ordem das chaves do dicionário"

Mais:
- Resposta **sensacional** no stackoverflow para entender melhor as diferenças entre dict (que a partir da versão 3.7 mantém a ordem de inserção como feature para todos os dicionários) e orderedDict: [Are dictionaries ordered in Python 3.6+?](https://stackoverflow.com/questions/39980323/are-dictionaries-ordered-in-python-3-6)
- Python docs https://docs.python.org/3/library/collections.html#collections.OrderedDict
- [CPython implementation - OrderedDict](https://github.com/python/cpython/blob/98ce7b107e6611d04dc35a4f5b02ea215ef122cf/Lib/collections/__init__.py#L94)

In [24]:
# dict vs. orderedDict
from collections import OrderedDict

print('AMBOS MANTÉM ORDEM DE INSERÇÃO')
std_dict = dict({'a': 1, 'c': 3, 'b': 2})
ord_dict = OrderedDict({'a': 1, 'c': 3, 'b': 2})

for key in std_dict:
    print(key, end=' ')
print()
for key in ord_dict:
    print(key, end=' ')
    
print('\n', '-'*10)

for key in reversed(std_dict):
    print(key, end=' ')
print()
for key in reversed(ord_dict):
    print(key, end=' ')
print('\n')

print('COMPARAÇÕES SÃO ORDER SENSITIVE EM ORDERED DICTS')
A = {'a': 1, 'b': 2, 'c': 3}
B = {'a': 1, 'c': 3, 'b': 2}
print('Comparação dict........:', A == B)
A = OrderedDict({'a': 1, 'c': 3, 'b': 2})
B = OrderedDict({'a': 1, 'b': 2, 'c': 3})
print('Comparação OrderedDict.:',A == B)

print()
print('ORDERED DICT TEM FUNCIONALIDADES A MAIS RELACIONADAS A ORDEM')
ord_dict.move_to_end('a') # move 'a' para o fim do dicionário
display(ord_dict)
std_dict.move_to_end('a') # AttributeError: 'dict' object has no attribute 'move_to_end'

AMBOS MANTÉM ORDEM DE INSERÇÃO
a c b 
a c b 
 ----------
b c a 
b c a 

COMPARAÇÕES SÃO ORDER SENSITIVE EM ORDERED DICTS
Comparação dict........: True
Comparação OrderedDict.: False

ORDERED DICT TEM FUNCIONALIDADES A MAIS RELACIONADAS A ORDEM


OrderedDict([('c', 3), ('b', 2), ('a', 1)])

AttributeError: 'dict' object has no attribute 'move_to_end'

### 3.2. ChainMap

Livro: "Armazena uma lista de mapeamentos que podem ser buscados como se fossem um só"

Mais:
- Texto muito esclarecedor (e que inspirou os exemplos daqui): [A pratical usage of chainmap in python](https://florimond.dev/blog/articles/2018/07/a-practical-usage-of-chainmap-in-python/)
- Stackoverflow: [What is the purpose of collections.ChainMap?](https://stackoverflow.com/questions/23392976/what-is-the-purpose-of-collections-chainmap)
- Python docs: https://docs.python.org/pt-br/3/library/collections.html#collections.ChainMap
- [CPython implementation - ChainMap](https://github.com/python/cpython/blob/98ce7b107e6611d04dc35a4f5b02ea215ef122cf/Lib/collections/__init__.py#L960)

In [31]:
# COMO FUNCIONA EM DICT - PARA COMPARAÇÃO
brinquedos = {'bola': 10, 'boneca': 30, 'quebra-cabeças': 1}
comidas = {'batata': 30, 'castanha': 100}

inventario = brinquedos.copy()
inventario.update(comidas)  
# também não funciona desempacotamento
# inventario = {**brinquedos, **comidas}

brinquedos.pop('bola')
brinquedos['boneca'] = 25
comidas['banana'] = 13

display(brinquedos)
display(inventario)

{'boneca': 25, 'quebra-cabeças': 1}

{'bola': 10, 'boneca': 30, 'quebra-cabeças': 1, 'batata': 30, 'castanha': 100}

In [33]:
# USANDO CHAIN MAP
from collections import ChainMap

brinquedos = {'bola': 10, 'boneca': 30, 'quebra-cabeças': 1}
comidas = {'batata': 30, 'castanha': 100}

inventario = ChainMap(brinquedos, comidas)

brinquedos.pop('bola')
brinquedos['boneca'] = 25

display(brinquedos)
display(inventario)

# facilmente iteramos por todos os itens do inventário
valor_total_inventario = 0
for item, valor in inventario.items():
    print(item, valor)
    valor_total_inventario += valor
print('Total: ', valor_total_inventario)

{'boneca': 25, 'quebra-cabeças': 1}

ChainMap({'boneca': 25, 'quebra-cabeças': 1}, {'batata': 30, 'castanha': 100})

batata 30
castanha 100
boneca 25
quebra-cabeças 1
Total:  156


### Porém!
Docs: "Buscas acontecem na cadeia de mapeamentos em ordem até que a chave seja encontrada. **Porém, opreação de escrita, update e deleções acontecem apenas no primeiro mapeamento**."

In [38]:
# OBS: deleções e edições só funcionam para os itens do PRIMEIRO mapeamento
map1 = {'a': 10, 'b': 30, 'c': 1}
map2 = {'c': 30, 'd': 100}

cadeia = ChainMap(map1, map2)

print('EDITANDO VALORES')
cadeia['b'] = 10
print("Valor de cadeia['d']: ", cadeia['d'])
cadeia['d'] = 10
print("Valor de cadeia['d']: ", cadeia['d'])
cadeia.update({'novos': 'valores'})
display(cadeia)  # todas as alterações foram feitas apenas no mapeamento referente a map1

print('REMOVENDO VALORES')
cadeia.pop('c')
display(cadeia)
cadeia.pop('c')  # KeyError: Key not found in the first mapping: 'c'

EDITANDO VALORES
Valor de cadeia['d']:  100
Valor de cadeia['d']:  10


ChainMap({'a': 10, 'b': 10, 'c': 1, 'd': 10, 'novos': 'valores'}, {'c': 30, 'd': 100})

REMOVENDO VALORES


ChainMap({'a': 10, 'b': 10, 'd': 10, 'novos': 'valores'}, {'c': 30, 'd': 100})

KeyError: "Key not found in the first mapping: 'c'"

In [40]:
# Então para que usar se não posso editar todos? -> caso de uso interessante

valores_config = {'app_name': 'super app'}
valores_default = {'app_name': 'nome default', 'qtd_exec': 5}

config = ChainMap(valores_config, valores_default)

print(f"""
App name...: {config['app_name']}
Execuções..: {config['qtd_exec']} 
""")


App name...: super app
Execuções..: 5 



### 3.3. Counter

Livro: "Um mapeamento que armazena um contador inteiro para cada chave. Atualizar uma chave existente faz o contador ser incrementado."

Mais:
- Python docs; https://docs.python.org/pt-br/3/library/collections.html#collections.Counter
- [CPython implementation - Counter](https://github.com/python/cpython/blob/98ce7b107e6611d04dc35a4f5b02ea215ef122cf/Lib/collections/__init__.py#L515)

In [41]:
# Exemplo do livro
from collections import Counter

ct = Counter('abracadabra')
display(ct)

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

In [61]:
# MÉTODOS LEGAIS

centro_zoonoses = Counter(gatos=4, doguinhos=8)
display(centro_zoonoses)

# É possível inicializar o contador com valores negativos
teste = Counter(gatos=-7, ratos=5)
display(teste)

# pegar uma chave inexistente retorna 0
print('Qtd gatos: ', centro_zoonoses['gatos'])
print('Qtd ratos: ', centro_zoonoses['ratos'])

# operadores de soma e subtração funcionam legal
centro_zoonoses_suburbio = Counter(gatos=7, ratos=5)
total_bichinhos = centro_zoonoses + centro_zoonoses_suburbio
display(total_bichinhos)

# PS: em operações, se o valor é zerado (ou fica menor que zero)
# a chave é removida
teste = Counter(gatos=7, ratos=-5, caes=4) - Counter(gatos=5, caes=10)
display(teste)

# possui uns métodos úteis
total_bichinhos.most_common(1)

Counter({'gatos': 4, 'doguinhos': 8})

Counter({'gatos': -7, 'ratos': 5})

Qtd gatos:  4
Qtd ratos:  0


Counter({'gatos': 11, 'doguinhos': 8, 'ratos': 5})

Counter({'gatos': 2})

[('gatos', 11)]

In [72]:
# Útil para fazer contagem de caracteres ou então um  bag of words
import this
import codecs
zen_of_python = codecs.encode(this.s, 'rot13')

bag_of_words = Counter(zen_of_python.split())

# Útil para pegar os mais comuns
display(bag_of_words.most_common(3))

# updates incrementa a contagem ao invés de dar replace no valor 
# (tem o contrário que é o método subtract)
bag_of_words.update({'is': 4})  # is é 14
bag_of_words.subtract({'is': 5})  # is é 9
bag_of_words.update(['better'])
display(bag_of_words.most_common(3))

# tudo pode ser corrompido
bag_of_words['better'] = 6
display(bag_of_words.most_common(3))

[('is', 10), ('better', 8), ('than', 8)]

[('is', 9), ('better', 9), ('than', 8)]

[('is', 9), ('than', 8), ('better', 6)]

### 3.4. UserDict

Livro: "É uma implementação Python pura de um mapeamento que funciona como um `dict` padrão. [...] quase sempre é mais fácil criar um novo tipo de mapeamento usando estendendo `UserDict` em vez de `dict`", isso porque "`dict` tem alguns atalhos de implementação que acabam nos forçando a sobrescrever métodos que podemos simplesmente herdar" _-> vamos ver mais sobre outros problemas em capítulos futuros do livro._

"Enquanto `OrderedDict`, `ChainMap` e `Counter` estão prontos para uso, `UserDict` foi concebido para que subclasses sejam criadas a partir dele".

More:
- Python docs: https://docs.python.org/pt-br/3/library/collections.html#collections.UserDict
- [CPython implementation - UserDict](https://github.com/python/cpython/blob/98ce7b107e6611d04dc35a4f5b02ea215ef122cf/Lib/collections/__init__.py#L1089) -> observe que `UserDict` não herda de `dict`, mas sim de `MutableMapping`! Porém, internamente, ele guarda uma instância de `dict` dentro de um atributo `data` onde guarda seus dados... isso evita recursões indesejadas em alguns métodos especiais.

In [ ]:
# EXEMPLO DO LIVRO
# Baseado no exemplo de implementação da classe StrKeyDict0 (mostrada no encontro anterior)

from collections import UserDict

class StrKeyDict(UserDict): # -> A classe herda de UserDict
    def __missing__(self, key):  # Igual a implementação de StrKeyDict0
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):  # Mais simples que a implementação de StrKeyDict0 porque
                                  # podemos supor que todas as chaves guardadas são str
        return str(key) in self.data
    
    def __setitem__(self, key, item):  # Mais fácil de sobrescrever, uma vez que 
                                       # estamos delegando para o atributo self.data (que é um dict)
        self.data[str(key)] = item

Como `UserDict` é uma subclasse de `MutableMapping`, os métodos restantes que tornam `StrKeyDict` um mapeamento completo são herdados de `UserDict`, `MutableMapping` ou `Mapping`. 

Os últimos têm diversos métodos concretos úteis, apesar de serem ABCs. Vale a pena dar uma olhada nos método a seguir:

#### MutableMapping.update ([código fonte no CPython](https://github.com/python/cpython/blob/master/Lib/_collections_abc.py#L848))
"Esse método poderoso pode ser chamado diretamente, mas é usado também por `__init__` para carregar a instância a partir de outros mapeamentos, iteráveis de pares `(key, value)` e argumentos nomeados. Por usar `self[key] = value` para adicionar itens, nossa implementação de `__setitem__` é chamada.

In [ ]:
# Código de MutabbleMapping.update no CPython (dia 26.08.2020)
def update(self, other=(), /, **kwds):
    ''' D.update([E, ]**F) -> None.  Update D from mapping/iterable E and F.
        If E present and has a .keys() method, does:     for k in E: D[k] = E[k]
        If E present and lacks .keys() method, does:     for (k, v) in E: D[k] = v
        In either case, this is followed by: for k, v in F.items(): D[k] = v
    '''
    if isinstance(other, Mapping):
        for key in other:
            self[key] = other[key]
    elif hasattr(other, "keys"):
        for key in other.keys():
            self[key] = other[key]
    else:
        for key, value in other:
            self[key] = value
    for key, value in kwds.items():
        self[key] = value

#### Mapping.get ([código fonte no CPython](https://github.com/python/cpython/blob/master/Lib/_collections_abc.py#L675))
Em `StrKeyDict0`, tivemos que codificar nosso próprio `get` para obter resultados consistentes com `__getitem__`, porém, no exemplo de `StrKeyDict`, herdamos `Mapping.get`, que é implementado exatamente como `StrKeyDict0.get` :)

In [11]:
# Código de Mapping.get no CPython (dia 26.08.2020)
def get(self, key, default=None):
    'D.get(k[,d]) -> D[k] if k in D, else d.  d defaults to None.'
    try:
        return self[key]
    except KeyError:
        return default